<img align="left" src = figs/logos/logo-IJCLab_v1.png height=40, style="padding: 10px"> 
<b>PhotoZ estimation with scikit learn Machine learning </b> <br>
Last verified to run on 2022-03-21 with LSST Science Pipelines release w_2021_49 <br>
Contact authors: Sylvie Dagoret-Campagne (DP0 Delegate) <br>
Target audience: DP0 delegates member <br>

**Credit:** Originally developed by Sylvie Dagoret-Campagne in the framework provided by Rubin DP0.1 (reference DP0.1 tutorials)

Acknowledgement: Melissa Graham, Leanne Guy, Alex Drlica-Wagner, Keith Bechtol, Grzegorz Madejski, Louise Edwards, and many others ..

# Learning Objectives : Find hyperparameters for optimal of scikit-learn PhotoZ estimator.


- Validation curves,
- Leaning curves,
- Bias and overfitting trade-off
- grid search cross-validation
- randoom search cross-validation


Note in the case of a live demo dedicated to PhotoZ, for which the time is limited, the Section 1 must be skipped. A selection flag  FLAG_SHOW_PHOTOMETRY_DETECTION = False is set accordingly. 


**Note:** : 
-All plots are made with Holoview.
- **Better select the maximum of CPU (4 CPU on RSP)**




### Imports

In [ ]:
# Import general python packages
import numpy as np
import re
import pandas as pd
import pickle
from pandas.testing import assert_frame_equal
import os
import errno
import shutil
import getpass
import datetime
# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst.geom as geom
import lsst.afw.coord as afwCoord
afwDisplay.setDefaultBackend('matplotlib')

#
from lsst import skymap

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.units.quantity import Quantity
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


# Bokeh for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize
from holoviews.operation.datashader import datashade, dynspread
from holoviews.plotting.util import process_cmap

import datashader as dsh


# Set the maximum number of rows to display from pandas
pd.set_option('display.max_rows', 20)


# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
#hv.extension('bokeh')
hv.extension('bokeh', 'matplotlib')

# Display bokeh plots inline in the notebook
output_notebook()

In [ ]:
# What versions of bokeh and holoviews nd datashader are we working with?
# This is important when referring to online documentation as
# APIs can change between versions.
print("Bokeh version: " + bokeh.__version__)
print("Holoviews version: " + hv.__version__)
print("Datashader version: " + dsh.__version__)

In [ ]:
#  What version of the Stack are we using?
! echo $IMAGE_DESCRIPTION
! eups list -s | grep lsst_distrib

In [ ]:
# allow for matplotlib to create inline plots in our notebook
%matplotlib inline
import matplotlib.pyplot as plt      # imports matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import warnings                      # imports the warnings library
import gc                            # imports python's garbage collector

# Ignore warnings
from astropy.units import UnitsWarning
warnings.simplefilter("ignore", category=UnitsWarning)

In [ ]:
# Set up some plotting defaults:

params = {'axes.labelsize': 28,
          'font.size': 24,
          'legend.fontsize': 14,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [18, 10],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

# Utility functions

## Tools to explain the expected detected magnitudes distributions

- taken from from https://github.com/ixkael/Photoz-tools

$$
p(m) = m^\alpha \exp\left( - \left(m/m_{max}\right)^\beta \right)
$$

In [ ]:
def p_mag(imag_grid,maglim):
    """   
    
    Model of magnitude distribution in photometric survey
    
    p_mag(imag_grid,maglim)
    from https://github.com/ixkael/Photoz-tools
    
    input args:
     - imag_grid : magnitude
     - maglim limit of magnitude
     
     return the probability of magnitude distribution
     
     THIS IS THE MODEL THAT MUST BE USED
    
    """

    # some parameters for prob(imagnitude)
    alpha = 15.0 
    beta = 2
    off=1

    # prob(imagnitude) distribution
    p_imag = imag_grid**alpha*np.exp(-(imag_grid/(maglim-off))**beta)
    p_imag /= p_imag.sum()
    return p_imag


In [ ]:
# imag errir distribution as function of mag limit, as in Rykoff et al
def imag_err(m, mlim):
    """
    from https://github.com/ixkael/Photoz-tools
    """
    
    a, b = 4.56, 1
    k = 1
    sigmadet = 5
    teff = np.exp(a + b * (mlim - 21.))
    F = 10**(-0.4*(m-22.5))
    Flim = 10**(-0.4*(mlim-22.5))
    Fnoise = (Flim/sigmadet)**2 * k * teff - Flim
    return 2.5/np.log(10) * np.sqrt( (1 + Fnoise/F) / (F*k*teff))

In [ ]:
def det_prob(imag_grid,maglim):
    """
    det_prob(imag_grid,maglim)
    
    Give detection probability of a magnitude
    from https://github.com/ixkael/Photoz-tools
    
    input arg:
    - imag_grid : magnitude grid
    - maglim limit of magnitude
    
    return histogram of magnitude probability
    
    THIS IS THE MODEL THAT MUST BE USED
    
    """
    pp_mag=p_mag(imag_grid,maglim)
    
    detprob = 1*pp_mag 
    ind = (imag_grid >= maglim - 0.4)
    #detprob[ind] *= ( 1 - scipy.special.erf((imag_grid[ind]-maglim+0.4)/0.4) )
    # detection probability looks like a sigmoid
    detprob[ind] *= np.exp( -0.5*((imag_grid[ind]-maglim+0.4)/0.2)**2)
    detprob /= detprob.sum() * (imag_grid[1]-imag_grid[0])
    return detprob



# Configurations and initialisation

## Holoview Configuration

In [ ]:
HV_CURVE_SINGLE_WIDTH  = 400
HV_CURVE_SINGLE_HEIGHT = 350
HV_CURVE_MULTI_WIDTH  = 300
HV_CURVE_MULTI_HEIGHT = 300
HV_CURVE_MULTI_FRAME_WIDTH = 300
HV_CURVE_MULTI_COLS   = 3

In [ ]:
NBINS_HISTO = 50

In [ ]:
HV_HISTO_SINGLE_WIDTH  = 600
HV_HISTO_SINGLE_HEIGHT = 600
HV_HISTO_MULTI_WIDTH  = 300
HV_HISTO_MULTI_HEIGHT = 300
HV_HISTO_MULTI_FRAME_WIDTH = 300
HV_HISTO_MULTI_COLS   = 3

In [ ]:
HV_IMAGE_SINGLE_WIDTH  = 400
HV_IMAGE_SINGLE_HEIGHT = 400
HV_IMAGE_SINGLE_FRAME_WIDTH = 600
HV_IMAGE_MULTI_WIDTH  = 400
HV_IMAGE_MULTI_HEIGHT = 400
HV_IMAGE_MULTI_FRAME_WIDTH = 300
HV_IMAGE_MULTI_COLS   = 3

## Notebook Configuration

#### Setup paths

In [ ]:
# username
myusername=getpass.getuser()

In [ ]:
# temporary folders if necessary
NBDIR       = 'photoz_part1'                           # relative path for this notebook output
TMPTOPDIR   = "/scratch"                               # always write some output in /scratch, never in user HOME 
TMPUSERDIR  = os.path.join(TMPTOPDIR,myusername)       # defines the path of user outputs in /scratch 
TMPNBDIR    = os.path.join(TMPUSERDIR,NBDIR)           # output path for this particular notebook

In [ ]:
# create user temporary directory
if not os.path.isdir(TMPUSERDIR):
    try:
        os.mkdir(TMPUSERDIR)
    except:
        raise OSError(f"Can't create destination directory {TMPUSERDIR}!" ) 

In [ ]:
# create this notebook temporary directory
if not os.path.isdir(TMPNBDIR):
    try:
        os.mkdir(TMPNBDIR)
    except:
        raise OSError(f"Can't create destination directory {TMPNBDIR}!" ) 

#### 1.2 Defines steering flags and parameters

The Output queries may be saved in files if requested. 
By defaults all the following flags are set False : no query output is saved in file.
To speed-up the demo, the presenter may keep some of those flags True.


In [ ]:
FLAG_WRITE_DATAFRAMEONDISK  = True                     # Select of query output will be saved on disk
FLAG_READ_DATAFRAMEFROMDISK = True                     # Select of the query can be red from disk if it exists
FLAG_CLEAN_DATAONDISK       = False                     # Select of the output queries saved in file will be cleaned at the end of the notebook

###  Retrieve source from catalog

#### Coadds-Magnitude cutoff for 10 years

In [ ]:
UMAX,GMAX,RMAX,IMAX,ZMAX,YMAX = 26.1, 27.4, 27.5, 26.8, 26.1,24.9

### Selection function in catalog

In [ ]:
# Define a function to build a query passing a coordinate and a search radius
def getQueryCircle(c: SkyCoord, r: Quantity) -> str:
    query = "SELECT obj.ra, obj.dec, obj.objectId, obj.extendedness, "\
            "obj.mag_u_cModel, obj.mag_g_cModel, obj.mag_r_cModel, "\
            "obj.mag_i_cModel, obj.mag_z_cModel, obj.mag_y_cModel, "\
            "obj.magerr_u_cModel, obj.magerr_g_cModel, obj.magerr_r_cModel, "\
            "obj.magerr_i_cModel, obj.magerr_z_cModel, obj.magerr_y_cModel, "\
            "truth.truth_type, truth.redshift, truth.match_objectId " \
            "FROM dp01_dc2_catalogs.object as obj " \
            "JOIN dp01_dc2_catalogs.truth_match as truth " \
            "ON truth.match_objectId = obj.objectId " \
            "WHERE CONTAINS(POINT('ICRS', obj.ra, obj.dec),"\
            "CIRCLE('ICRS', " + str(c.ra.value) + ", " + str(c.dec.value) + ", " \
            + str(r.to(u.deg).value) + " )) = 1 " \
            "AND obj.good = 1 "  \
            "AND truth.match_objectid >= 0 " \
            "AND truth.is_good_match = 1 " \
            "AND truth.truth_type = 1 " \
            "AND obj.mag_u_cModel - 5*obj.magerr_u_cModel < " +str(UMAX) +" "\
            "AND obj.mag_g_cModel - 5*obj.magerr_g_cModel < " +str(GMAX) +" "\
            "AND obj.mag_r_cModel - 5*obj.magerr_r_cModel < " +str(RMAX) +" "\
            "AND obj.mag_i_cModel - 5*obj.magerr_i_cModel < " +str(IMAX) +" "\
            "AND obj.mag_z_cModel - 5*obj.magerr_z_cModel < " +str(ZMAX) +" "\
            "AND obj.mag_y_cModel - 5*obj.magerr_y_cModel < " +str(YMAX)
    return query

## Configuration and initialisation of the Rubin TAP Service client

In [ ]:
# Get an instance of the TAP service
service = get_tap_service()
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

In [ ]:
# Define a reference position on the sky  for a square seach
c1 = SkyCoord(ra=62.0*u.degree, dec=-40.*u.degree, frame='icrs')
size = 1.0 * u.deg

In [ ]:
query = getQueryCircle(c1, size)

In [ ]:
query

In [ ]:
filename_result=f'cat_photozpart1_result.pkl'
fullfilename_result=os.path.join(TMPNBDIR,filename_result)

# Selection flags

- put boolean flags here to avoid execution of some sections

In [ ]:
# Show plots on redshift distribution
FLAG_SHOW_TRUE_REDSHIFT_DISTRIB = False

In [ ]:
# Show plots to check the photometry selected for photoz
# For a pure demo on photoZ, this section can be skipped
FLAG_SHOW_PHOTOMETRY_DETECTION = False

# START HERE

- To speed up the database query, especially if you run this notebook in the context of a DP0 demo public session,
you can copy my data file **cat_photozpart1_result.pkl** from **/scratch/sylvielsstfr/photoz_part1/cat_photozpart1_result.pkl** to your path **/scratch/yourusername/photoz_part1**

- check the variable **FLAG_READ_DATAFRAMEFROMDISK=True**
- your username is given by **yourusername=getpass.getuser()**

## Read input data

In [ ]:
if FLAG_READ_DATAFRAMEFROMDISK and os.path.exists(fullfilename_result):
    sql_result = pd.read_pickle(fullfilename_result)
else:
    job = service.submit_job(query)
    job.run()
    job.wait(phases=['COMPLETED', 'ERROR'])
    print('Job phase is', job.phase)
    #sql_result = job.fetch_result().to_table().to_pandas()
    
    
    # Create and submit the job. This step does not run the query yet
    job = service.submit_job(query)
    # Get the job URL
    print('Job URL is', job.url)

    # Get the job phase. It will be pending as we have not yet started the job
    print('Job phase is', job.phase)
    
    # Run the job. You will see that the the cell completes executing,
    # even though the query is still running
    job.run()
    
    # Use this to tell python to wait for the job to finish if
    # you don't want to run anything else while waiting
    # The cell will continue executing until the job is finished
    job.wait(phases=['COMPLETED', 'ERROR'])
    print('Job phase is', job.phase)
    
    # A usefull funtion to raise an exception if there was a problem with the query
    job.raise_if_error()
    
    # Once the job completes successfully, you can fetch the results
    async_data = job.fetch_result()
    
    sql_result = async_data.to_table().to_pandas()
    
    
if FLAG_WRITE_DATAFRAMEONDISK:
    sql_result.to_pickle(fullfilename_result)
    

In [ ]:
sql_result.head()

In [ ]:
data = sql_result

In [ ]:
# for shorter names
data.rename(columns={"mag_u_cModel": "mag_u", "mag_g_cModel": "mag_g","mag_r_cModel": "mag_r",
                     "mag_i_cModel": "mag_i", "mag_z_cModel": "mag_z","mag_y_cModel": "mag_y",
                     "magerr_u_cModel": "magerr_u", "magerr_g_cModel": "magerr_g","magerr_r_cModel": "magerr_r",
                     "magerr_i_cModel": "magerr_i", "magerr_z_cModel": "magerr_z","magerr_y_cModel": "magerr_y",
                    },inplace=True)

In [ ]:
# output directory where the extracted catalog is temporary saved
! ls -l $TMPNBDIR

In [ ]:
# map truth_type
data['truth_type']=data['truth_type'].map({1: 'galaxy', 2: 'star', 3: 'SNe'})

#### drop objects that are not galaxies

In [ ]:
#drop objects that are not galaxies
data.drop(data.loc[data['truth_type'] != 'galaxy' ].index, inplace=True)

In [ ]:
# drop NA
data = data.dropna()

In [ ]:
len(data)

### add color

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
data["umg"]=data["mag_u"]- data["mag_g"]
data["gmr"]=data["mag_g"]- data["mag_r"]
data["rmi"]=data["mag_r"]- data["mag_i"]
data["imz"]=data["mag_i"]- data["mag_z"]
data["zmy"]=data["mag_z"]- data["mag_y"]

# Ckeck input data

## Redshifts distribution

In [ ]:
if FLAG_SHOW_TRUE_REDSHIFT_DISTRIB:
    (z_bin, count) = np.histogram(data.redshift, bins=NBINS_HISTO)
    z_distribution = hv.Histogram(z_bin, count).opts(title=f"redshift distribution",color='darkblue', 
    xlabel='redshift', fontscale=1.2,
    height=HV_HISTO_SINGLE_HEIGHT-100, width=HV_HISTO_SINGLE_WIDTH,tools=['hover'])
    
    z_distribution

## Magnitudes distribution

### Principle of photodetection

Study of a simple model to find the expected magnitude distribution including photodetection bias:

- blue curve : the true magnitude distribution
- green curve : the detected magnitude distribution
- red : detection threshold


In [ ]:
mymag_grid = np.linspace(17,30,100)
mymaglim = 27
mymag0 = 17
mymag1 = 30
prob_mag_tab = p_mag(mymag_grid,mymaglim)
dens_mag_tab = prob_mag_tab/np.sum(prob_mag_tab)/(mymag_grid[1]-mymag_grid[0])
det_prob_tab =  det_prob(mymag_grid,mymaglim)

In [ ]:
curve_opts = dict(
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '8pt'},
                height=HV_CURVE_SINGLE_HEIGHT, width=HV_CURVE_SINGLE_WIDTH+100,tools=['hover']
               )  

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    curve_magdist = hv.Curve(zip(mymag_grid,dens_mag_tab),label="true mag").opts(**curve_opts).opts(color="blue") 
    curve_maglim = hv.VLine(mymaglim,label="detection threshold").opts(color="red")
    curve_prob = hv.Curve(zip(mymag_grid,det_prob_tab),label="detected mag").opts(**curve_opts).opts(color="green") 

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    layout = (curve_magdist * curve_prob * curve_maglim).opts(legend_position='top_left',xlabel="magnitude",ylabel="probability",title="magnitude density distribution")
    layout

### Magnitude distribution before cutoff at detection threshold

- The catalog sources have been extracted $5\sigma$ above the cutoff. This allow the user to refine the expected cutoff inside each band 

In [ ]:
# Detection cutiff definition
maglim_u = hv.VLine(UMAX).opts(color="magenta")
maglim_g = hv.VLine(GMAX).opts(color="magenta")
maglim_r = hv.VLine(RMAX).opts(color="magenta")
maglim_i = hv.VLine(IMAX).opts(color="magenta")
maglim_z = hv.VLine(ZMAX).opts(color="magenta")
maglim_y = hv.VLine(YMAX).opts(color="magenta")

In [ ]:
histo_opts = dict(
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '8pt'},
                height=HV_HISTO_MULTI_HEIGHT, width=HV_HISTO_MULTI_WIDTH,tools=['hover']
               )  

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    mag_bin, count = np.histogram(data.mag_u, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magu = hv.Histogram(mag_bin, count).opts(title=f"mag U",xlabel="U",color='blue').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_g, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magg = hv.Histogram(mag_bin, count).opts(title=f"mag G",xlabel="G",color='green').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_r, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magr = hv.Histogram(mag_bin, count).opts(title=f"mag R",xlabel="R",color='red').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_i, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magi = hv.Histogram(mag_bin, count).opts(title=f"mag I",xlabel="I",color='orange').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_z, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magz = hv.Histogram(mag_bin, count).opts(title=f"mag Z",xlabel="Z",color='grey').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_y, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magy = hv.Histogram(mag_bin, count).opts(title=f"mag Y",xlabel="Y",color='black').opts(**histo_opts)

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    layout1 = h_magu * maglim_u + h_magg * maglim_g + h_magr * maglim_r + h_magi *  maglim_i + h_magz *  maglim_z + h_magy *  maglim_y
    layout1.cols(HV_HISTO_MULTI_COLS)

### Apply Photometric detection

- The magnitudes in DC2 correcpond to average magnitudes and their photometric error. In the telescope, the detection correspond to the realisation of one sample of these distribution.
Then the cutoff will be applied on detected magnitudes. (But the evaluation of photoz algorithm will be applied on the average magnitudes in the catalog)

#### randomisation of magnitude with photometric errors

- gaussian distribution should be applied to flux, not magnitudes. Assume central limit theorem applies on magnitudes as well. This could be checked or corrected.

In [ ]:
def photodet_mag(mag,errmag):
    return np.random.normal(loc=mag, scale=errmag)

In [ ]:
data['mag_u_det'] = data.apply(lambda x:  photodet_mag(x['mag_u'], x['magerr_u']), axis=1)
data['mag_g_det'] = data.apply(lambda x:  photodet_mag(x['mag_g'], x['magerr_g']), axis=1)
data['mag_r_det'] = data.apply(lambda x:  photodet_mag(x['mag_r'], x['magerr_r']), axis=1)
data['mag_i_det'] = data.apply(lambda x:  photodet_mag(x['mag_i'], x['magerr_i']), axis=1)
data['mag_z_det'] = data.apply(lambda x:  photodet_mag(x['mag_z'], x['magerr_z']), axis=1)
data['mag_y_det'] = data.apply(lambda x:  photodet_mag(x['mag_y'], x['magerr_y']), axis=1)

#### Selection on detected magnitudes

In [ ]:
def photodet_select(mu,mg,mr,mi,mz,my):
    return (mu>17) and (mu < UMAX) and (mg < GMAX) and (mr < RMAX) and (mi < IMAX) and (mz < ZMAX) and (my < YMAX)

In [ ]:
data['selected'] = data.apply(lambda x:  photodet_select(x['mag_u_det'], x['mag_g_det'], x['mag_r_det'], x['mag_i_det'],x['mag_z_det'], x['mag_y_det'] ), axis=1)

In [ ]:
data = data[data["selected"]]

In [ ]:
data = data.drop('selected', axis=1)

In [ ]:
data = data.dropna()

### Distribution of average magnitudes after photometric detection

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    mag_bin, count = np.histogram(data.mag_u, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magu = hv.Histogram(mag_bin, count).opts(title=f"mag U",xlabel="U",color='blue').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_g, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magg = hv.Histogram(mag_bin, count).opts(title=f"mag G",xlabel="G",color='green').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_r, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magr = hv.Histogram(mag_bin, count).opts(title=f"mag R",xlabel="R",color='red').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_i, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magi = hv.Histogram(mag_bin, count).opts(title=f"mag I",xlabel="I",color='orange').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_z, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magz = hv.Histogram(mag_bin, count).opts(title=f"mag Z",xlabel="Z",color='grey').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_y, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magy = hv.Histogram(mag_bin, count).opts(title=f"mag Y",xlabel="Y",color='black').opts(**histo_opts)

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    layout2 = h_magu * maglim_u + h_magg * maglim_g + h_magr * maglim_r + h_magi *  maglim_i + h_magz *  maglim_z + h_magy *  maglim_y
    layout2.cols(HV_HISTO_MULTI_COLS)

### Distribution of detected magnitudes after photometric detection

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    mag_bin, count = np.histogram(data.mag_u_det, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magud = hv.Histogram(mag_bin, count).opts(title=f"mag U",xlabel="U",color='blue').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_g_det, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_maggd = hv.Histogram(mag_bin, count).opts(title=f"mag G",xlabel="G",color='green').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_r_det, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magrd = hv.Histogram(mag_bin, count).opts(title=f"mag R",xlabel="R",color='red').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_i_det, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magid = hv.Histogram(mag_bin, count).opts(title=f"mag I",xlabel="I",color='orange').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_z_det, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magzd = hv.Histogram(mag_bin, count).opts(title=f"mag Z",xlabel="Z",color='grey').opts(**histo_opts)

    mag_bin, count = np.histogram(data.mag_y_det, bins=NBINS_HISTO,range=(mymag0,mymag1))
    h_magyd = hv.Histogram(mag_bin, count).opts(title=f"mag Y",xlabel="Y",color='black').opts(**histo_opts)

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    layout3 = h_magud * maglim_u + h_maggd * maglim_g + h_magrd * maglim_r + h_magid *  maglim_i + h_magzd *  maglim_z + h_magyd *  maglim_y
    layout3.cols(HV_IMAGE_MULTI_COLS)

### Detected magnitude vs true magnitude

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    huudet, xhuz, yhuz=np.histogram2d(data.mag_u,data.mag_u_det,bins=(50, 50),range=[[16,28],[16,28]])
    hggdet, xhgz, yhgz=np.histogram2d(data.mag_g,data.mag_g_det,bins=(50, 50),range=[[16,28],[16,28]])
    hrrdet, xhrz, yhrz=np.histogram2d(data.mag_r,data.mag_r_det,bins=(50, 50),range=[[16,28],[16,28]])
    hiidet, xhiz, yhiz=np.histogram2d(data.mag_i,data.mag_i_det,bins=(50, 50),range=[[16,28],[16,28]])
    hzzdet, xhzz, yhzz=np.histogram2d(data.mag_z,data.mag_z_det,bins=(50, 50),range=[[16,28],[16,28]])
    hyydet, xhyz, yhyz=np.histogram2d(data.mag_y,data.mag_y_det,bins=(50, 50),range=[[16,28],[16,28]])

In [ ]:
img_opts = dict(
                #height=600, width=700, 
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '8pt'},
                #colorbar=True, toolbar='right', show_grid=True,
                #aspect='equal',
                frame_width= HV_HISTO_MULTI_FRAME_WIDTH,
                show_grid=True ,
                #ylabel="mag",
                tools=['hover','undo','redo','zoom_in','zoom_out'],
                #tools=[myhover,'crosshair'],
               )  

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    fhuudet=np.flipud(huudet.T)
    fhggdet=np.flipud(hggdet.T)
    fhrrdet=np.flipud(hrrdet.T)
    fhiidet=np.flipud(hiidet.T)
    fhzzdet=np.flipud(hzzdet.T)
    fhyydet=np.flipud(hyydet.T)
    img_uud=hv.Image(fhuudet, bounds=(16,16,28,28) ).opts(cmap="Blues",title="magU det vs magU true ",xlabel="U (mag)",ylabel="U (mag)").opts(**img_opts)
    img_ggd=hv.Image(fhggdet, bounds=(16,16,28,28) ).opts(cmap="Greens",title="magG det vs magG true",xlabel="G (mag)",ylabel="G (mag)").opts(**img_opts)
    img_rrd=hv.Image(fhrrdet, bounds=(16,16,28,28) ).opts(cmap="Reds",title="magR det vs magR true",xlabel="R (mag)",ylabel="R (mag)").opts(**img_opts)
    img_iid=hv.Image(fhiidet, bounds=(16,16,28,28) ).opts(cmap="Oranges",title="magI det vs magI true",xlabel="I (mag)",ylabel="I (mag)").opts(**img_opts)
    img_zzd=hv.Image(fhzzdet, bounds=(16,16,28,28) ).opts(cmap="Greys",title="magZ det vs magZ true",xlabel="Z (mag)",ylabel="Z (mag)").opts(**img_opts)
    img_yyd=hv.Image(fhyydet, bounds=(16,16,28,28) ).opts(cmap="GnBu",title="magY det vs magY true",xlabel="Y (mag)",ylabel="Y (mag)").opts(**img_opts)

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    layout = img_uud + img_ggd + img_rrd + img_iid + img_zzd + img_yyd
    layout.cols(HV_IMAGE_MULTI_COLS)

### Average magnitude vs redshift

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    huz, xhuz, yhuz=np.histogram2d(data.redshift,data.mag_u,bins=(50, 50),range=[[0,3],[16,26]])
    hgz, xhgz, yhgz=np.histogram2d(data.redshift,data.mag_g,bins=(50, 50),range=[[0,3],[16,26]])
    hrz, xhrz, yhrz=np.histogram2d(data.redshift,data.mag_r,bins=(50, 50),range=[[0,3],[16,26]])
    hiz, xhiz, yhiz=np.histogram2d(data.redshift,data.mag_i,bins=(50, 50),range=[[0,3],[16,26]])
    hzz, xhzz, yhzz=np.histogram2d(data.redshift,data.mag_z,bins=(50, 50),range=[[0,3],[16,26]])
    hyz, xhyz, yhyz=np.histogram2d(data.redshift,data.mag_y,bins=(50, 50),range=[[0,3],[16,26]])

In [ ]:
img_opts = dict(
                #height=600, width=700, 
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '8pt'},
                #colorbar=True, toolbar='right', show_grid=True,
                #aspect='equal',
                frame_width= HV_HISTO_MULTI_FRAME_WIDTH,
                show_grid=True ,
                xlabel="redshift",
                #ylabel="mag",
                tools=['hover','undo','redo','zoom_in','zoom_out'],
                #tools=[myhover,'crosshair'],
               )  

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    fhuz=np.flipud(huz.T)
    fhgz=np.flipud(hgz.T)
    fhrz=np.flipud(hrz.T)
    fhiz=np.flipud(hiz.T)
    fhzz=np.flipud(hzz.T)
    fhyz=np.flipud(hyz.T)
    img_uz=hv.Image(fhuz, bounds=(0,16,3,26) ).opts(cmap="Blues",title="magU vs reshift",ylabel="U (mag)").opts(**img_opts)
    img_gz=hv.Image(fhgz, bounds=(0,16,3,26) ).opts(cmap="Greens",title="magG vs reshift",ylabel="G (mag)").opts(**img_opts)
    img_rz=hv.Image(fhrz, bounds=(0,16,3,26) ).opts(cmap="Reds",title="magR vs reshift",ylabel="R (mag)").opts(**img_opts)
    img_iz=hv.Image(fhiz, bounds=(0,16,3,26) ).opts(cmap="Oranges",title="magI vs reshift",ylabel="I (mag)").opts(**img_opts)
    img_zz=hv.Image(fhzz, bounds=(0,16,3,26) ).opts(cmap="Greys",title="magZ vs reshift",ylabel="Z (mag)").opts(**img_opts)
    img_yz=hv.Image(fhyz, bounds=(0,16,3,26) ).opts(cmap="GnBu",title="magY vs reshift",ylabel="Y (mag)").opts(**img_opts)

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    layout = img_uz + img_gz + img_rz + img_iz + img_zz + img_yz
    layout.cols(HV_IMAGE_MULTI_COLS)

### Average Color vs redshift

In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    humgz, xhumgz, yhumgz=np.histogram2d(data.redshift,data.umg,bins=(50, 50),range=[[0,3],[-0.5,2.]])
    hgmrz, xhgmrz, yhgmrz=np.histogram2d(data.redshift,data.gmr,bins=(50, 50),range=[[0,3],[-0.5,2.]])
    hrmiz, xhrmiz, yhrmiz=np.histogram2d(data.redshift,data.rmi,bins=(50, 50),range=[[0,3],[-0.5,2.]])
    himzz, xhimzz, yhimzz=np.histogram2d(data.redshift,data.imz,bins=(50, 50),range=[[0,3],[-0.5,2.]])
    hzmyz, xhzmyz, yhzmyz=np.histogram2d(data.redshift,data.zmy,bins=(50, 50),range=[[0,3],[-0.5,2.]])

    fhumgz=np.flipud(humgz.T)
    fhgmrz=np.flipud(hgmrz.T)
    fhrmiz=np.flipud(hrmiz.T)
    fhimzz=np.flipud(himzz.T)
    fhzmyz=np.flipud(hzmyz.T)

    img_umgz=hv.Image(fhumgz, bounds=(0,-0.5,3,2.) ).opts(cmap="Blues",title="U - G vs reshift",ylabel="U-G (mag)").opts(**img_opts)
    img_gmrz=hv.Image(fhgmrz, bounds=(0,-0.5,3,2.) ).opts(cmap="Greens",title="G - R vs reshift",ylabel="G-R (mag)").opts(**img_opts)
    img_rmiz=hv.Image(fhrmiz, bounds=(0,-0.5,3,2.) ).opts(cmap="Reds",title="R - I vs reshift",ylabel="R-I (mag)").opts(**img_opts)
    img_imzz=hv.Image(fhimzz, bounds=(0,-0.5,3,2.) ).opts(cmap="Oranges",title="I - Z vs reshift",ylabel="I-Z (mag)").opts(**img_opts)
    img_zmyz=hv.Image(fhzmyz, bounds=(0,-0.5,3,2.) ).opts(cmap="Greys",title="Z - Y vs reshift",ylabel="Z-Y (mag)").opts(**img_opts)


In [ ]:
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    layout = img_umgz + img_gmrz + img_rmiz + img_imzz + img_zmyz 
    layout.cols(HV_IMAGE_MULTI_COLS)

Because photo-z's rely on the flux integrated in broad filters, they are more sensitive to broad, dramatic features of the SED. This means that the location of these breaks provides a lot of information for photo-z's. 
- In particular, the Balmer and 4000 angstrom breaks are in the wavelength range of the LSST filters up to a redshift of ~1.4, 
- and locating these breaks with the LSST filters provides good leverage for photo-z's (see e.g. Kalmbach et al. 2020 and Malz et al. 2021). 
- Note that while the Balmer break leaves the LSST filters around z=1.4, 
- the Lyman break doesn't enter the wavelength range of the LSST filters until about z=2.5 :  This gap in redshift coverage contributes to the degradation of photo-z's at high redshift.

- Note that in high-redshift galaxies, photo-z estimators might confuse the Lyman break for the Balmer break. This contributes to the physical degeneracies discussed above in "What makes photo-z's hard?"

In [ ]:
from PIL import Image
if FLAG_SHOW_PHOTOMETRY_DETECTION: 
    img= Image.open("figs/sdss/plot_sdss_filters_2.png")
    img = img.resize((500, 400), Image.ANTIALIAS)
    img

# Machine Learning for Photo-Z estimation

## Requirements from LSST Science book:
(https://www.lsst.org/sites/default/files/docs/sciencebook/SB_3.pdf)

Photometric redshifts for LSST will be applied and calibrated over the redshift range $0 < z < 4$
for galaxies to $r  \simeq 27.5$. 
For the majority of science cases, such as weak lensing and BAO, a subset
of galaxies with $i < 25.3$ will be used. For this high S/N gold standard subset over the
redshift interval, $0 < z < 3$, the photometric redshift requirements are:

- The root-mean-square scatter in photometric redshifts, $ \sigma_z/(1+z)$, must be smaller than 0.05, with a goal of 0.02.
- The fraction of $3\sigma $  outliers at all redshifts must be below 10%.
- The bias in $e_z = (z_{photo}−z_{spec})/(1+z_{spec})$ must be below 0.003 (or 0.01 for combined,analyses of weak lensing and baryon acoustic oscillations); 
- The uncertainty in  $\sigma_z/(1+z)$ must also be known to similar accuracy.



### other definitions

- **the photo-z accuracy is the absolute value of the difference between the true and photometric redshifts**.

-  **the photo-z uncertainty is the standard deviation of the true redshifts** 

## Utility functions


- from DE School IV, University of Oxford, July 18, 2016 :  **Jeff Newman - photometric redshifts for LSST**

The tools for PhotoZ evaluation are givenin the notebook and also described in the LSST science book (Performance Chapter) 

### Performances Evaluation lines

In [ ]:
#A function that we will call a lot: makes the zphot/zspec plot and calculates key statistics
def plot_lines(zmin=0,zmax=3,zstep=0.05,slope=0.15):
    
    x = np.arange(zmin,zmax,zstep)
    outlier_upper = x + slope*(1+x)
    outlier_lower = x - slope*(1+x)

    curv_bisect=hv.Curve(zip(x,x)).opts(color="red") 
    curv_outupper=hv.Curve(zip(x,outlier_upper)).opts(color="red",line_dash='dashed') 
    curv_outlower=hv.Curve(zip(x,outlier_lower)).opts(color="red",line_dash='dashed') 
    
    layout = curv_bisect * curv_outupper * curv_outlower
    return layout
    
    

In [ ]:
plot_lines()

### Statistic lines

In [ ]:
def get_stats(z_spec,z_phot,slope=0.15):
    """
    input : 
       - z_spec : spectroscopic redshift or true redshift
       - z_phot : photo-z reedshift
       - slope : slope of line defining the outliers  3 x sigma_z with sigma_z = 5%, so slope = 3 x 0.05 = 0.15 
    """
    
    mask = np.abs((z_phot - z_spec)/(1 + z_spec)) > slope
    notmask = ~mask 
    
    # Standard Deviation of the predicted redshifts compared to the data:
    #-----------------------------------------------------------------
    std_result = np.std((z_phot - z_spec)/(1 + z_spec), ddof=1)
    print('Standard Deviation: %6.4f' % std_result)
    

    # Normalized MAD (Median Absolute Deviation):
    #------------------------------------------
    nmad = 1.48 * np.median(np.abs((z_phot - z_spec)/(1 + z_spec)))
    print('Normalized MAD: %6.4f' % nmad)

    # Percentage of delta-z > 0.15(1+z) outliers:
    #-------------------------------------------
    eta = np.sum(np.abs((z_phot - z_spec)/(1 + z_spec)) > 0.15)/len(z_spec)
    print('Delta z >0.15(1+z) outliers: %6.3f percent' % (100.*eta))
    
    # Median offset (normalized by (1+z); i.e., bias:
    #-----------------------------------------------
    bias = np.median(((z_phot - z_spec)/(1 + z_spec)))
    sigbias=std_result/np.sqrt(0.64*len(z_phot))
    print('Median offset: %6.3f +/- %6.3f' % (bias,sigbias))
    
    
     # overlay statistics with titles left-aligned and numbers right-aligned
    stats_txt = '\n'.join([
        'NMAD  = {:0.2f}'.format(nmad),
        'STDEV = {:0.2f}'.format(std_result),
        'BIAS  = {:0.2f}'.format(bias),
        'ETA   = {:0.2f}'.format(eta)
    ])
    
    
    return nmad,std_result,bias,eta,stats_txt
    

## START ML here

### Prepare and Feature

Because we want to estimate the performance of photoz estimator itself, not the total performance including intrinsic redshift fluctuations. Thus only average magnitudes data will be used : detected magnitude are dropped. 

In [ ]:
target = data["redshift"]

In [ ]:
features = data[["mag_u","mag_g","mag_r","mag_i","mag_z","mag_y"]]

- Total number of samples to split in training, validation and test dataset

In [ ]:
Ntot = len(target)
Ntot

### Split in training / test set

- speed of the notebook must be tuned with the training sample size

#### number of samples to be used in training

- depending on the required speed of the demo 

In [ ]:
Ntrain = 5000
Ntest = Ntot-Ntrain

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Test fraction
test_sample_size_fraction=Ntest/Ntot
test_sample_size_fraction

In [ ]:
# adapt the train dataset size according required running time 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_sample_size_fraction, random_state=0)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
features

** Note**
- because the model fit (training) may be long, we should limit the training dataset size for this demo.

## Regressors definitions

### Regularized Linear model

- Instead of using the LinearRegressor, we start by using the regularized Ridge regressor with the alpha parameter setting the regularization.
- Linear model features should be always normalized,
- For non linearities, we include the possibility to develop the model as a polynomial of features

Scikit-Learn offer to define pipelines of tasks in an easy way:
- PolynomialFeatures() task extend features dataset in powers of thos features up to a power degree,
- StandardScaler() preprocess the features to normalize them,
- Ridge is the regularized version of the LinearRegressor

In [ ]:
from sklearn import set_config
set_config(display="diagram")

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

ridge_regressor = make_pipeline(PolynomialFeatures(degree=5), StandardScaler(),Ridge(alpha=0.05))

In [ ]:
ridge_regressor 

### RandomForest regressor

- RandomForest regressor is an Enssemble regressor of type Bagging (Bootstrap and Aggregation).  

- RandomForest regressor combines the regression of multiple decision tree regressors fitted in parallel 
on bootstrapped samples from the training sample.

- Each individual decision tree is deep, meaning they individually overfit the bootstrapped samples. 

- The aggregation of the overfitting parallel decision tree model reduce the over-fitting.
- For Random Forest, each Decision Tree node feature are drawn randomly. This reduce the error correlation of the various trees.
- From this caracteristics, RandomForest is expected to be one of the best non-linear regressor on column-tabulated datasets.


- RandomForst includes a number of hyper-parameters.
- We use the hyper-parameters chosen byJeff Newmann for the DE-School at Oxford 2016.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
randomforest_regressor = RandomForestRegressor(n_estimators = 50, max_depth = 30, max_features = 'auto')

### Gradient Boosting Regressor

The INRIA MOOC (2022) on scikit-Learn ( Machine learning in Python with scikit-learn: https://lms.fun-mooc.fr/courses/course-v1:inria+41026+session02/info)
recommend the histogram-binned version of GradientBoostingRegressor, expected to have a good balance between underfitting and overfitting.

- Boosting Regressor perform shallow Decision trees (underfitting) fit sequencially. Among them, Gradient Boosting Regressor are expected to avoid overfitting. Among them the Histogram Grandient Boosting regressor is expected to run faster    

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
#from sklearn.preprocessing import KBinsDiscretizer
#discretizer = KBinsDiscretizer(n_bins=64, encode="ordinal", strategy="quantile")
#histogram_gradient_boosting_regressor = make_pipeline(discretizer, HistGradientBoostingRegressor(max_iter=30))
histogram_gradient_boosting_regressor =  HistGradientBoostingRegressor(max_iter=30)

## Evaluation metrics

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.metrics import make_scorer
scoring = {'r2': make_scorer(r2_score),'mae': make_scorer(mean_absolute_error),'mse': make_scorer(mean_squared_error)}

## Use of cross-validation

- We use cross-validation to select a sub-sample of galaxies from the complete training sample and train the model with this subset.
- This sub-sampling is repeated several times (n_split=5).

The interest of this multi-subsampling is to have a set of almost similar but slightly different fitted models from which we can derive several predictions for a test sample, thus an average predicted value and its variation (or a PDF)

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
from sklearn.model_selection import ShuffleSplit

In [ ]:
# We don't know of the galaxies are ordered by redshift or come randomly. Thus we activate a pre-random-shuffling in the training dataset. 
cv = ShuffleSplit(n_splits=5, test_size=.80, random_state=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

## Ridge model

- The cross_validate function performs fit on n_splits models from n_splits random subsamples
- The smaple is previously randomized
- The evaluation metric is given by the scoring (The INRIA MOOC use this coring="neg_mean_absolute_error"),
- The n_splits fitted models are retuned (to be able to make n_split prediction for a single test sample)

### Validation curve

In [ ]:
# get parameters names
# ridge_regressor.get_params()

#### Validation curve on the regularisation hyperparameter alpha

In [ ]:
%%time
from sklearn.model_selection import validation_curve

the_degree = 2
the_title  = f"Ridge regressor : Validation curve with degree = {the_degree}"
ridge_regressor = make_pipeline(PolynomialFeatures(degree=the_degree), StandardScaler(),Ridge(alpha=0.05))

alpha_grid = np.array([ 0.0001, 0.0005, 0.001, 0.005,  0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10., 50. ])
train_scores, test_scores = validation_curve(
    ridge_regressor, X_train, y_train, param_name="ridge__alpha", param_range=alpha_grid,
    cv=cv, scoring="neg_mean_squared_error", n_jobs=2)
train_errors, test_errors = np.sqrt(-train_scores.mean(axis=1)), np.sqrt(-test_scores.mean(axis=1))

In [ ]:
curv_train=hv.Curve(zip(np.log10(alpha_grid) , train_errors),label="train").opts(color="blue",xlabel="log(alpha)",ylabel="error",xlim=(-4,2)) 
curv_test=hv.Curve(zip(np.log10(alpha_grid) , test_errors),label="test").opts(color="red",xlabel="log(alpha)",ylabel="error",xlim=(-4,2)) 
layout = (curv_train * curv_test).opts(width=600,height=300,title=the_title,legend_position='bottom_right',ylim=(0,1.1*test_errors.max()))

In [ ]:
layout

##### comments on validation curve for alpha:
- high alpha give high bias on both training and testing validation curve (right of the figure)
- low alpha means no regularisation. Training and testing validation curves are diverging. Error on training are decreasing (training overfitting) while error on testing is increasing (left of the figure).  
- A good parameter for alpha is between [0.1 , 3]

#### Validation curve on the  hyperparameter : degree of the polynomial

In [ ]:
%%time
from sklearn.model_selection import validation_curve

the_alpha = 0.01
the_degree = 2
the_title  = f"Ridge regressor : Validation curve with alpha = {the_alpha}"
ridge_regressor = make_pipeline(PolynomialFeatures(degree=the_degree), StandardScaler(),Ridge(alpha=the_alpha))

degree_grid = np.arange(1,12)
train_scores, test_scores = validation_curve(
    ridge_regressor, X_train, y_train, param_name="polynomialfeatures__degree", param_range=degree_grid,
    cv=cv, scoring="neg_mean_squared_error", n_jobs=2)
train_errors, test_errors = np.sqrt(-train_scores.mean(axis=1)), np.sqrt(-test_scores.mean(axis=1))

In [ ]:
curv_train=hv.Curve(zip(degree_grid , train_errors),label="train").opts(color="blue",xlabel="degree",ylabel="error") 
curv_test=hv.Curve(zip(degree_grid , test_errors),label="test").opts(color="red",xlabel="degree",ylabel="error") 
layout = (curv_train * curv_test).opts(width=600,height=300,title=the_title,legend_position='bottom_right',ylim=(0,1.1*test_errors.max()))

In [ ]:
layout

##### comments on validation curve for degree:
- low degree means too simple model (left of the figure) means high bias for both training and testing validation curve.
- high degree means means too complex model (right of the figure). Training and testing validation curves are diverging. Error on training are decreasing (training overfitting) while error on testing is increasing (left of the figure).  
- A good parameter for degree is 2

### Learning curve

In [ ]:
Ntrain = 10000
Ntest = Ntot-Ntrain
# Test fraction
test_sample_size_fraction=Ntest/Ntot

# adapt the train dataset size according required running time 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_sample_size_fraction, random_state=0)

In [ ]:
train_sizes=np.arange(50,Ntrain//5,50)

In [ ]:
from sklearn.model_selection import learning_curve

the_alpha = 0.1
the_degree = 2
the_title  = f"Ridge regressor : Learning curve : degree={the_degree} , alpha = {the_alpha}"

ridge_regressor = make_pipeline(PolynomialFeatures(degree=the_degree), StandardScaler(),Ridge(alpha=the_alpha))

results = learning_curve(ridge_regressor, X_train, y_train, train_sizes=train_sizes, cv=cv,
    scoring="neg_mean_squared_error", n_jobs=2)

#results = learning_curve(ridge_regressor, X_train, y_train, cv=cv,
#    scoring="neg_mean_squared_error", n_jobs=2)

train_size, train_scores, test_scores = results[:3]
# Convert the scores into errors
train_errors, test_errors = np.sqrt(-train_scores.mean(axis=1)),np.sqrt( -test_scores.mean(axis=1))

In [ ]:
curv_train=hv.Curve(zip(train_sizes , train_errors),label="train").opts(color="blue",xlabel="n_train",ylabel="error") 
curv_test=hv.Curve(zip(train_sizes , test_errors),label="test").opts(color="red",xlabel="n_train",ylabel="error") 
layout = (curv_train * curv_test).opts(width=600,height=300,legend_position='bottom_right',title=the_title,ylim=(0,1.1*test_errors.max()))

In [ ]:
layout

##### comments on leaning curve:

- Linear model require few samples for learning.
- Training and testing learning curve converge at about n_sample=1000, with no error gap between the two curve.
- This means the model is good without under-fitting nor overfitting.


### Grid search Cross Validation

- The goal is to find the optimum pair of hyper parameters by a grid search

In [ ]:
# list the parameter names
#ridge_regressor.get_params()

In [ ]:
# define the grid search of hyper parameters
param_grid = {'polynomialfeatures__degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'ridge__alpha': [0.001, .01, .1, 1., 10. ]}

In [ ]:
grid_search_ridge = GridSearchCV(estimator=ridge_regressor, param_grid=param_grid, scoring="neg_mean_squared_error" ,n_jobs=2)

#### Limitation of the number of samples 

In [ ]:
Ntrain = 1000
Ntest = Ntot-Ntrain
# Test fraction
test_sample_size_fraction=Ntest/Ntot

# adapt the train dataset size according required running time 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_sample_size_fraction, random_state=0)

#### Fit of the best hyperparameters 

In [ ]:
%%time
t1 = datetime.datetime.now()
grid_search_ridge.fit(X_train, y_train)
t2 = datetime.datetime.now()
deltat = (t2-t1).total_seconds() 
print(f"Ridge GridSearchCV : elapsed time {deltat:.2f} sec")

In [ ]:
print(f"The best parameters found are: {grid_search_ridge.best_params_}")
print(f"The best score: {grid_search_ridge.best_score_}")

In [ ]:
best_estimator = grid_search_ridge.best_estimator_

In [ ]:
df_cv_results = pd.DataFrame(grid_search_ridge.cv_results_)

In [ ]:
df_cv_results.rename(columns={"param_polynomialfeatures__degree":"degree","param_ridge__alpha":"alpha"},inplace=True)

In [ ]:
df_cv_results = df_cv_results[["degree","alpha","mean_test_score","std_test_score","rank_test_score"]]

In [ ]:
df_cv_results["mean_test_score"] = np.sqrt(- df_cv_results["mean_test_score"])

In [ ]:
df_cv_results.sort_values(by="rank_test_score", axis=0, ascending=True)

In [ ]:
# Nested CV with parameter optimization
# Too long for a live demo !!!
# cv_results = cross_validate(grid_search_ridge, X_train, y_train, cv=3, n_jobs=2, scoring = "neg_mean_squared_error", return_estimator = True)

### estimation

In [ ]:
# Choose one of the n_splits model, but all predictions for all estimators could be calculated (average and rms)
y_pred = best_estimator.predict(X_test)

### Performances

In [ ]:
coords = zip(y_test,y_pred)
points = hv.Points(coords).opts(tools=['box_select', 'lasso_select'])

In [ ]:
nmad,std_result,bias,eta,stats_txt1 = get_stats(y_test.values,y_pred)

In [ ]:
# Create a holoviews object to hold and plot data
# Create the linked streams instance
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

# Apply the datashader
p1 = dynspread(datashade(points, cmap="Viridis"))
p1 = p1.opts(width=HV_HISTO_SINGLE_WIDTH, height=HV_HISTO_SINGLE_HEIGHT,
    padding=0.05, show_grid=True,
    xlim=(0, 3), ylim=(0, 3.0),
    xlabel="z-spec", ylabel="z-phot",title="Optimum Ridge Regressor (grid CV)")

In [ ]:
p1 * plot_lines() * hv.Text(0.5, 2.5, stats_txt1)

#### performance metrics

In [ ]:
r2  = r2_score(y_pred,y_test)
mae = mean_absolute_error(y_pred,y_test)
mse = mean_squared_error(y_pred,y_test)

In [ ]:
msg_r2   = f"R2 score : \t\t {r2:.3f}"
msg_mae  = f"MAE mean absolute error : \t {mae:.3f}"
msg_rmsq = f"Root MSE error : \t\t {np.sqrt(mse):.3f}"

In [ ]:
print(msg_r2)
print(msg_mae)
print(msg_rmsq)

### RandomizedSearchCV 

In [ ]:
from scipy.stats import loguniform


class loguniform_int:
    """Integer valued version of the log-uniform distribution"""
    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        """Random variable sample"""
        return self._distribution.rvs(*args, **kwargs).astype(int)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'polynomialfeatures__degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'ridge__alpha': loguniform(0.001, 10),
}

random_search_ridge  = RandomizedSearchCV(
    ridge_regressor, param_distributions=param_distributions, n_iter=20,
    cv=5, verbose=1,
)


In [ ]:
%%time
t1 = datetime.datetime.now()
random_search_ridge.fit(X_train, y_train)
t2 = datetime.datetime.now()
deltat = (t2-t1).total_seconds() 
print(f"Ridge RandomSearchCV : elapsed time {deltat:.2f} sec")

In [ ]:
print(f"The best parameters found are: {random_search_ridge.best_params_}")

In [ ]:
print(f"The best score: {random_search_ridge.best_score_}")

In [ ]:
best_estimator = random_search_ridge.best_estimator_

In [ ]:
df_cv_results = pd.DataFrame(random_search_ridge.cv_results_)

In [ ]:
df_cv_results.rename(columns={"param_polynomialfeatures__degree":"degree","param_ridge__alpha":"alpha"},inplace=True)

In [ ]:
df_cv_results = df_cv_results[["degree","alpha","mean_test_score","std_test_score","rank_test_score"]]

In [ ]:
df_cv_results.sort_values(by="rank_test_score", axis=0, ascending=True)

### Estimation

In [ ]:
y_pred = best_estimator.predict(X_test)

### Performances

In [ ]:
coords = zip(y_test,y_pred)
points = hv.Points(coords).opts(tools=['box_select', 'lasso_select'])

In [ ]:
nmad,std_result,bias,eta,stats_txt1b = get_stats(y_test.values,y_pred)

In [ ]:
# Create a holoviews object to hold and plot data
# Create the linked streams instance
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

# Apply the datashader
p1b = dynspread(datashade(points, cmap="Viridis"))
p1b = p1b.opts(width=HV_HISTO_SINGLE_WIDTH, height=HV_HISTO_SINGLE_HEIGHT,
    padding=0.05, show_grid=True,
    xlim=(0, 3), ylim=(0, 3.0),
    xlabel="z-spec", ylabel="z-phot",title="Optimum Ridge Regressor (random CV)")

In [ ]:
p1b * plot_lines() * hv.Text(0.5, 2.5, stats_txt1b)

#### performance metrics

In [ ]:
r2  = r2_score(y_pred,y_test)
mae = mean_absolute_error(y_pred,y_test)
mse = mean_squared_error(y_pred,y_test)

In [ ]:
msg_r2   = f"R2 score : \t\t {r2:.3f}"
msg_mae  = f"MAE mean absolute error : \t {mae:.3f}"
msg_rmsq = f"Root MSE error : \t\t {np.sqrt(mse):.3f}"

In [ ]:
print(msg_r2)
print(msg_mae)
print(msg_rmsq)

## Random Forest

### Select smaller training sample

- faster model fit

In [ ]:
Ntrain = 5000
Ntest = Ntot-Ntrain
test_sample_size_fraction=Ntest/Ntot
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_sample_size_fraction, random_state=0)

### Validation curves

In [ ]:
the_n_estimators = 50
the_max_depth = 5

In [ ]:
randomforest_regressor = RandomForestRegressor(n_estimators = the_n_estimators, max_depth = the_max_depth, max_features = 'auto')

In [ ]:
%%time
from sklearn.model_selection import validation_curve


n_estimators_grid = range(0,50,5)
max_depth_grid = range(1,30)

train_scores, test_scores = validation_curve(
    randomforest_regressor, X_train, y_train, param_name="n_estimators", param_range=n_estimators_grid ,
    cv=cv, scoring="neg_mean_squared_error", n_jobs=2)

train_errors, test_errors = np.sqrt(-train_scores.mean(axis=1)), np.sqrt(-test_scores.mean(axis=1))


In [ ]:
the_title = f"Random Forest regressor : Validation curve : max_depth = {the_max_depth}"
curv_train=hv.Curve(zip(n_estimators_grid  , train_errors),label="train").opts(color="blue",xlabel="n_estimators",ylabel="error") 
curv_test=hv.Curve(zip(n_estimators_grid  , test_errors),label="test").opts(color="red",xlabel="n_estimators",ylabel="error") 
layout = (curv_train * curv_test).opts(width=600,height=300,title=the_title,ylim=(0,test_errors.max()*1.1))

In [ ]:
layout

##### comments on validation curve for n_estimators:
- Error not very sensitive with n_estimators
- Small overfit on training sample 

In [ ]:
%%time
from sklearn.model_selection import validation_curve

n_estimators_grid = range(0,50,5)
max_depth_grid = range(1,30)

train_scores, test_scores = validation_curve(
    randomforest_regressor, X_train, y_train, param_name="max_depth", param_range=max_depth_grid ,
    cv=cv, scoring="neg_mean_squared_error", n_jobs=2)

train_errors, test_errors = np.sqrt(-train_scores.mean(axis=1)), np.sqrt(-test_scores.mean(axis=1))

In [ ]:
the_title = f"Random Forest regressor : Validation curve : n_estimators = {the_n_estimators}"
curv_train=hv.Curve(zip(max_depth_grid  , train_errors),label="train").opts(color="blue",xlabel="max_depth",ylabel="error") 
curv_test=hv.Curve(zip(max_depth_grid  , test_errors),label="test").opts(color="red",xlabel="max_depth",ylabel="error") 
layout = (curv_train * curv_test).opts(width=600,height=300,title=the_title,ylim=(0,test_errors.max()*1.1))

In [ ]:
layout

##### comments on validation curve for max_depth:
- at low max_depth, high bias on training and test set
- above few max_depth , overfitting on training set

### Learning curve

In [ ]:
Ntrain = 10000
Ntest = Ntot-Ntrain
# Test fraction
test_sample_size_fraction=Ntest/Ntot

# adapt the train dataset size according required running time 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_sample_size_fraction, random_state=0)

In [ ]:
train_sizes=np.arange(10,Ntrain//5,50)

In [ ]:
from sklearn.model_selection import learning_curve

randomforest_regressor = RandomForestRegressor(n_estimators = the_n_estimators, max_depth = the_max_depth, max_features = 'auto')

results = learning_curve(randomforest_regressor, X_train, y_train, train_sizes=train_sizes, cv=cv,
    scoring="neg_mean_squared_error", n_jobs=2)
#results = learning_curve(randomforest_regressor, X_train, y_train, cv=cv,
#    scoring="neg_mean_squared_error", n_jobs=2)


train_size, train_scores, test_scores = results[:3]
# Convert the scores into errors
train_errors, test_errors = np.sqrt(-train_scores.mean(axis=1)), np.sqrt(-test_scores.mean(axis=1))

In [ ]:
the_title = f"Random Forest regressor : Learning curve  n_estimators={the_n_estimators} , max_depth = {the_max_depth}"
curv_train=hv.Curve(zip(train_sizes , train_errors),label="train").opts(color="blue",xlabel="n_train",ylabel="error") 
curv_test=hv.Curve(zip(train_sizes , test_errors),label="test").opts(color="red",xlabel="n_train",ylabel="error") 
layout = (curv_train * curv_test).opts(width=700,height=300,title=the_title,ylim=(0,test_errors.max()*1.1))

In [ ]:
layout

##### comments on leaning curve:

- Training and testing learning curve converge at about n_sample=1000, with a residual error gap between the two curve.
- This means the model is slightly overfitting

### Grid Search cross- Validation

In [ ]:
Ntrain = 2000
Ntest = Ntot-Ntrain
# Test fraction
test_sample_size_fraction=Ntest/Ntot

# adapt the train dataset size according required running time 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_sample_size_fraction, random_state=0)

In [ ]:
#randomforest_regressor.get_params() 

In [ ]:
param_grid = {'n_estimators': [10, 20, 30, 40, 50], 'max_depth': [2,5,10,15,20,25,30]}

In [ ]:
grid_search_randomforest = GridSearchCV(estimator=randomforest_regressor, param_grid=param_grid, n_jobs=2)

In [ ]:
%%time
t1 = datetime.datetime.now()
grid_search_randomforest.fit(X_train, y_train)
t2 = datetime.datetime.now()
deltat = (t2-t1).total_seconds() 
print(f"Random Forest GridSearchCV : elapsed time {deltat:.2f} sec")

In [ ]:
print(f"The best parameters found are: {grid_search_randomforest.best_params_}")
print(f"The best score: {grid_search_randomforest.best_score_}")

In [ ]:
best_estimator = grid_search_randomforest.best_estimator_

In [ ]:
df_cv_results = pd.DataFrame(grid_search_randomforest.cv_results_)

In [ ]:
df_cv_results.rename(columns={"param_max_depth":"max_depth","param_n_estimators":"n_estimators"},inplace=True)

In [ ]:
df_cv_results = df_cv_results[["max_depth","n_estimators","mean_test_score","std_test_score","rank_test_score"]]

In [ ]:
df_cv_results.sort_values(by="rank_test_score", axis=0, ascending=True)

### Estimation

In [ ]:
# Choose one of the n_splits model, but all predictions for all estimators could be calculated (average and rms)
y_pred = best_estimator.predict(X_test)

### Performances

In [ ]:
nmad,std_result,bias,eta,stats_txt2= get_stats(y_test.values,y_pred)

In [ ]:
coords = zip(y_test,y_pred)
points = hv.Points(coords).opts(tools=['box_select', 'lasso_select'])

In [ ]:
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

# Apply the datashader
p2 = dynspread(datashade(points, cmap="Viridis"))
p2 = p2.opts(width=HV_HISTO_SINGLE_WIDTH, height=HV_HISTO_SINGLE_HEIGHT,
    padding=0.05, show_grid=True,
    xlim=(0, 3), ylim=(0, 3.0),
    xlabel="z-spec", ylabel="z-phot",title="Optimum Random Forest Regressor")

In [ ]:
p2 * plot_lines() *  hv.Text(0.5, 2.5, stats_txt2)

#### Performance metrics in scikit learn

In [ ]:
r2  = r2_score(y_pred,y_test)
mae = mean_absolute_error(y_pred,y_test)
mse = mean_squared_error(y_pred,y_test)

In [ ]:
msg_r2   = f"R2 score : \t\t {r2:.3f}"
msg_mae  = f"MAE mean absolute error : \t {mae:.3f}"
msg_rmsq = f"Root MSE error : \t\t {np.sqrt(mse):.3f}"

In [ ]:
print(msg_r2)
print(msg_mae)
print(msg_rmsq)

## Histogram Gradient Boosting regressor

- No particular optimisation of hyper parameters performed here 

### Validation curve on max_iter

In [ ]:
Ntrain = 5000
Ntest = Ntot-Ntrain
test_sample_size_fraction=Ntest/Ntot
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_sample_size_fraction, random_state=0)

In [ ]:
histogram_gradient_boosting_regressor = HistGradientBoostingRegressor(random_state=0)
max_iter = [1, 2, 5, 10, 20, 50, 100, 200, 500]
train_scores_hgbdt, test_scores_hgbdt = validation_curve(
    histogram_gradient_boosting_regressor, X_train, y_train, param_name="max_iter", param_range=max_iter, scoring="neg_mean_squared_error",cv=cv, n_jobs=2
)

In [ ]:
train_error = np.sqrt(-train_scores_hgbdt.mean(axis=1))
test_error  = np.sqrt(-test_scores_hgbdt.mean(axis=1))

In [ ]:
curv_train=hv.Curve(zip(max_iter  , train_errors),label="train").opts(color="blue",xlabel="max_iter",ylabel="error") 
curv_test=hv.Curve(zip(max_iter  , test_errors),label="test").opts(color="red",xlabel="max_iter",ylabel="error") 
layout = (curv_train * curv_test).opts(width=600,height=300,title="Histogram gradient boosting : Validation curve",ylim=(0,test_errors.max()))

In [ ]:
layout

### Learning curve 

In [ ]:
Ntrain = 10000
Ntest = Ntot-Ntrain
# Test fraction
test_sample_size_fraction=Ntest/Ntot

# adapt the train dataset size according required running time 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_sample_size_fraction, random_state=0)

In [ ]:
train_sizes=np.arange(10,Ntrain//5,50)

In [ ]:
train_size

In [ ]:
the_max_iter = 30

from sklearn.model_selection import learning_curve

histogram_gradient_boosting_regressor = HistGradientBoostingRegressor(max_iter=the_max_iter,random_state=0)

results = learning_curve(histogram_gradient_boosting_regressor, X_train, y_train, train_sizes=train_sizes, cv=cv,
    scoring="neg_mean_squared_error", n_jobs=2)
#results = learning_curve(histogram_gradient_boosting_regressor, X_train, y_train, cv=cv,
#    scoring="neg_mean_squared_error", n_jobs=2)

train_size, train_scores, test_scores = results[:3]
# Convert the scores into errors
train_errors, test_errors = np.sqrt(-train_scores.mean(axis=1)), np.sqrt(-test_scores.mean(axis=1))

In [ ]:
the_title = f"Histogram Gradient Boosting : Learning curve  : max_iter = {the_max_iter}"
curv_train=hv.Curve(zip(train_sizes , train_errors),label="train").opts(color="blue",xlabel="n_train",ylabel="error") 
curv_test=hv.Curve(zip(train_sizes , test_errors),label="test").opts(color="red",xlabel="n_train",ylabel="error") 
layout = (curv_train * curv_test).opts(width=700,height=300,title=the_title,ylim=(0,test_errors.max()*1.1))

In [ ]:
layout

##### comments on leaning curve:

- Training and testing learning curve converge very slowly a with a residual error gap between the two curve.
- This means the model is slightly overfitting

In [ ]:
#discretizer = KBinsDiscretizer(n_bins=64, encode="ordinal", strategy="quantile")
#histogram_gradient_boosting_regressor = make_pipeline(discretizer, HistGradientBoostingRegressor(max_iter=30))
histogram_gradient_boosting_regressor = HistGradientBoostingRegressor(max_iter=20)

In [ ]:
param_grid = {'max_iter': range(10,60,10)}

In [ ]:
grid_search_gradient_boosting = GridSearchCV(estimator=histogram_gradient_boosting_regressor, param_grid=param_grid, n_jobs=2)

### Limit the number ov samples for optimisation

In [ ]:
Ntrain = 2000
Ntest = Ntot-Ntrain
# Test fraction
test_sample_size_fraction=Ntest/Ntot

# adapt the train dataset size according required running time 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_sample_size_fraction, random_state=0)

### CV grid search training 

In [ ]:
%%time
# We simply use the fit method, not the cross_validate to accelerate the demo 
t1 = datetime.datetime.now()
grid_search_gradient_boosting.fit(X_train,y_train) 
t2 = datetime.datetime.now()
deltat = (t2-t1).total_seconds() 
print(f"Histogram Gradient Boosting GridSearchCV : elapsed time {deltat:.2f} sec")

In [ ]:
print(f"The best parameters found are: {grid_search_gradient_boosting.best_params_}")
print(f"The best score: {grid_search_gradient_boosting.best_score_}")

In [ ]:
best_estimator=grid_search_gradient_boosting.best_estimator_

### Estimation

In [ ]:
y_pred =  best_estimator.predict(X_test)

### Performances

In [ ]:
coords = zip(y_test,y_pred)
points = hv.Points(coords).opts(tools=['box_select', 'lasso_select'])

In [ ]:
nmad,std_result,bias,eta,stats_txt3 = get_stats(y_test.values,y_pred)

In [ ]:
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

# Apply the datashader
p3 = dynspread(datashade(points, cmap="Viridis"))
p3 = p3.opts(width=HV_HISTO_SINGLE_WIDTH, height=HV_HISTO_SINGLE_HEIGHT,
    padding=0.05, show_grid=True,
    xlim=(0, 3), ylim=(0, 3.0),
    xlabel="z-spec", ylabel="z-phot",title="Optimum Histogram Gradient Boosting Regressor")

In [ ]:
p3 * plot_lines() *  hv.Text(0.5, 2.5, stats_txt3)

#### Performance metrics in scikit learn

In [ ]:
r2  = r2_score(y_pred,y_test)
mae = mean_absolute_error(y_pred,y_test)
mse = mean_squared_error(y_pred,y_test)

In [ ]:
msg_r2   = f"R2 score : \t\t {r2:.3f}"
msg_mae  = f"MAE mean absolute error : \t {mae:.3f}"
msg_rmsq = f"Root MSE error : \t\t {np.sqrt(mse):.3f}"

In [ ]:
print(msg_r2)
print(msg_mae)
print(msg_rmsq)

# Future work : continue optimisation

The bias is OK, but the shape and the scatter is not that good.

- Play with other hyper-parameters to improve Random Forest and Hist Gradient Boosting estimators,
- Try other estimators SVM, Neural networks,...

# Clean the output directory 

In [ ]:
if FLAG_CLEAN_DATAONDISK:
    if os.path.isdir(TMPNBDIR):
        try:
            shutil.rmtree(TMPNBDIR)
        except OSError as e:
            print("Error: %s : %s" % (TMPNBDIR, e.strerror)) 